In [ ]:
import tensorflow as tf
from tensorflow.python.client import timeline
import numpy as np
np.tau = 2*np.pi
import matplotlib.pyplot as plt
import time
from concurrent.futures import ThreadPoolExecutor

# Lamb-Oseen setup

In [ ]:
Nx = Ny = 60
xmin,xmax = -1,1
ymin,ymax = -1,1
X, Y = np.mgrid[xmin:xmax:Nx*1j,ymin:ymax:Ny*1j]


Gamma = 20
r0 = 0.5
nu = 0.004
P0=0
rho = 1060

rmax = np.sqrt(1.25643)*r0

def LO(t):
    V_th = Gamma/(np.tau*np.sqrt(X**2+Y**2))*(1-np.exp(-(X**2+Y**2)/(r0**2+4*nu*t)))
    V_x = V_th*Y/np.sqrt(X**2+Y**2)
    V_y = -V_th*X/np.sqrt(X**2+Y**2)
    return np.stack((V_x,V_y),axis=0)



theta = np.linspace(0,np.tau)
wall_pos = np.stack((0.5+0.49*np.cos(theta),0.5+0.49*np.sin(theta)),axis=-1)
wall_normals = np.stack((np.cos(theta),
                        np.sin(theta)),axis=-1)
vx,vy = LO(0)

plt.quiver(wall_pos[...,0],wall_pos[...,1],wall_normals[...,0],wall_normals[...,1])
plt.imshow(np.hypot(vx,vy),extent=[0,1,0,1])
plt.show()

# Divergence kernel

In [ ]:
lins = np.linspace(-2,2,5)

# Kernel values at grid positions
kernel = [np.exp(-_x**2) for _x in lins]
kernel_div = [np.exp(-_x**2)*-2*_x for _x in lins]

div_kernel = np.dstack((np.prod(np.meshgrid(kernel_div,kernel,indexing='ij'),axis=0),
                        np.prod(np.meshgrid(kernel,kernel_div,indexing='ij'),axis=0)))

plt.subplot(211)
plt.scatter(lins,np.exp(-lins**2))
plt.scatter(lins,np.exp(-lins**2)*2*-lins)
plt.subplot(223)
plt.imshow(np.prod(np.meshgrid(kernel_div,kernel,indexing='ij'),axis=0).T)
plt.subplot(224)
plt.imshow(np.prod(np.meshgrid(kernel,kernel_div,indexing='ij'),axis=0).T)
plt.show()


# Computation

In [ ]:
# Helper function to hide the nitty gritty spline details
def compute_2d_splines(x):
    indices = tf.cast(x/dx+1,tf.int32)
    coeff = tf.gather_nd(C, XY[None]+indices[...,None,None,:])
    offsets = (tf.mod(x/dx,1)[...,None,None,:]-tf.cast(XY[None],tf.float32))*dx
    f = tf.exp(-tf.reduce_sum((offsets/sc)**2,axis=-1))
    return tf.reduce_sum(f[...,None]*coeff,axis=(1,2))


# Setup graph variables
tf.reset_default_graph()
dx = 0.15
sc = dx

# Compute shape directly instead
n = np.mgrid[-dx:1+1.5*dx:dx,-dx:1+1.5*dx:dx].transpose(1,2,0)

C = tf.Variable(np.zeros(n.shape[:-1]+(2,)),dtype=tf.float32)
Div = tf.constant(div_kernel[...,None],dtype=tf.float32)

x = tf.placeholder(np.float32)
v = tf.placeholder(np.float32)
w = tf.placeholder(np.float32)

wall = tf.placeholder(np.float32)
wall_n = tf.placeholder(np.float32)

# Common
r = tf.range(-1,3,dtype=tf.int32)
XY = tf.transpose(tf.meshgrid(r,r))

# Data fit
model = compute_2d_splines(x)

# Divergence fit
model_div = tf.nn.conv2d(C[None],Div,[1, 1, 1, 1],'VALID')

# Wall fit
model_wall = tf.reduce_sum(compute_2d_splines(wall)*wall_n,axis=-1)

# Setup loss terms
Q0 = tf.reduce_mean(w*tf.square(model-v))
Q1 = tf.reduce_mean(tf.square(C))
Q2 = tf.reduce_mean(tf.square(model_div))
Q3 = tf.reduce_mean(tf.square(model_wall))


k1 = 0.1
k2 = 10
k3 = 10

loss = Q0+k1*Q1+k2*Q2+k3*Q3

# Training step
train = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

# Initialize variables
init = tf.global_variables_initializer()

# Execute train steps

In [ ]:
lo = LO(0).transpose(1,2,0)
lo += 5*np.random.normal(size=lo.shape)
pos = np.mgrid[0:0.999:Nx*1j,0:0.999:Ny*1j].transpose(1,2,0)

weights = np.zeros((Nx,Ny,2))#np.random.rand(Nx,Ny)
weights[:Nx//2,:Ny//2,0] = 1
weights[Nx//2:,Ny//2:,1] = 1

mask = np.asarray(np.sum(weights,axis=-1),dtype=np.bool)
N = len(pos[mask])
losses=[]
with tf.Session() as sess:
    sess.run(init)  

    def trainer():
        choice = np.random.randint(0,N,size=500)
        losses.append(sess.run([loss,train],feed_dict={x:pos[mask][choice].reshape(-1,2),v:lo[mask][choice].reshape(-1,2),w:weights[mask][choice].reshape(-1,2),
                                                       wall: wall_pos, wall_n: wall_normals}))[0]
    with ThreadPoolExecutor(max_workers=16) as executor:
        for i in range(300):
            executor.submit(trainer)


    c = sess.run(model,feed_dict={x:pos.reshape(-1,2)})
plt.plot(losses)
plt.show()

# Visualize results

In [ ]:
plt.subplot(231)
plt.imshow(c.reshape(Nx,Ny,2)[...,0],clim=(-5,5))
plt.title('Vx')
plt.subplot(232)
plt.imshow(LO(0).transpose(1,2,0)[...,0],clim=(-5,5))
plt.title('Vx GT')
plt.subplot(233)
plt.imshow(1/weights[...,0]*lo[...,0],clim=(-10,10))
plt.title('Vx input')

plt.subplot(234)
plt.imshow(c.reshape(Nx,Ny,2)[...,1],clim=(-5,5))
plt.title('Vz')
plt.subplot(235)
plt.imshow(LO(0).transpose(1,2,0)[...,1],clim=(-5,5))
plt.title('Vz GT')
plt.subplot(236)
plt.imshow(1/weights[...,1]*lo[...,1],clim=(-10,10))
plt.title('Vz input')
plt.tight_layout()
plt.savefig('VFM.pdf')

plt.show()

# VFM

In [ ]:
# Setup graph variables
tf.reset_default_graph()
C = tf.Variable(np.zeros((Nx,Ny,2)),dtype=tf.float32)

div = np.asarray([[1,-2,1],[1,-2,1],[1,-2,1]])
lap = np.asarray([[0.5, 1.0, 0.5],[1.0, -6., 1.0],[0.5, 1.0, 0.5]])

D = tf.constant(np.dstack((div,div.T))[...,None],dtype=np.float32)
L = tf.constant(lap[...,None,None],dtype=np.float32)

idx = tf.placeholder(np.int32)
v = tf.placeholder(np.float32)
w = tf.placeholder(np.float32)

# Data fit
model = tf.gather_nd(C,idx)

# Divergence
model_div = tf.nn.conv2d(C[None],D,[1, 1, 1, 1],'VALID')

# Laplacian
model_lap = tf.square(tf.nn.conv2d(C[None,...,:1],L,[1, 1, 1, 1],'VALID'))+tf.square(tf.nn.conv2d(C[None,...,1:],L,[1, 1, 1, 1],'VALID'))

# Setup loss terms
Q0 = tf.reduce_mean(w*tf.square(model-v))
Q1 = tf.reduce_mean(tf.square(C))
Q2 = tf.reduce_mean(tf.square(model_div))
Q3 = tf.reduce_mean(model_lap)



k1 = 0.01
k2 = 50
k3 = 50

loss = Q0+k1*Q1+k2*Q2+k3*Q3

# Training step
train = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

# Initialize variables
init = tf.global_variables_initializer()

In [ ]:
lo = LO(0).transpose(1,2,0)
lo += 10*np.random.normal(size=lo.shape)
weights = np.ones((Nx,Ny,2))#np.random.rand(Nx,Ny)


with tf.Session() as sess:
    sess.run(init)
    losses = []
    for i in range(2000):
        xs,ys = np.meshgrid(np.arange(Nx),np.arange(Ny),indexing='ij')
        losses.append(sess.run([loss,train], feed_dict={idx:np.stack((xs.ravel(),ys.ravel()),axis=-1),v:lo.reshape(-1,2),w:weights.reshape(-1,2)})[0])
    c = sess.run(C)
    plt.plot(losses[len(losses)//2:])
    plt.show()

In [ ]:
plt.subplot(231)
plt.imshow(c.reshape(Nx,Ny,2)[...,0],clim=(-5,5))
plt.title('Vx')
plt.subplot(232)
plt.imshow(LO(0).transpose(1,2,0)[...,0],clim=(-5,5))
plt.title('Vx GT')
plt.subplot(233)
plt.imshow(1/weights[...,0]*lo[...,0],clim=(-10,10))
plt.title('Vx input')

plt.subplot(234)
plt.imshow(c.reshape(Nx,Ny,2)[...,1],clim=(-5,5))
plt.title('Vz')
plt.subplot(235)
plt.imshow(LO(0).transpose(1,2,0)[...,1],clim=(-5,5))
plt.title('Vz GT')
plt.subplot(236)
plt.imshow(1/weights[...,1]*lo[...,1],clim=(-10,10))
plt.title('Vz input')
plt.tight_layout()
plt.savefig('VFM.pdf')

plt.show()